A neural network consist of 3 cnn layers and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [ ]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D,Conv1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [ ]:
len(labels)

78

In [ ]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [ ]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [ ]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [ ]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [ ]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [ ]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [ ]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [ ]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [ ]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    # print(cnn.shape)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    # print(cnn.shape)
    cnn = MaxPooling1D(3)(cnn)
    # print(cnn.shape)
    

    cnn2 = Conv1D(1, 
                 kernel_size=(3),
                 activation='relu')(cnn)
    # print(cnn2.shape)
    # cnn1 =  Lambda(lambda x: K.sum(x, axis=3))(cnn1)
    # print(cnn2.shape)
    cnn2 = MaxPooling1D(3)(cnn2)
    # print(cnn2.shape)

    # cnn2 = Lambda(lambda x: K.sum(x, axis=2))(cnn2)
    # print(cnn2.shape)

    cnn3 = Conv1D(1, 
                 kernel_size=(3),
                 activation='relu')(cnn2)
    print(cnn3.shape)
    # cnn1 =  Lambda(lambda x: K.sum(x, axis=3))(cnn1)
    print(cnn3.shape)
    cnn3 = MaxPooling1D(3)(cnn3)
    print(cnn3.shape)

    cnn3 = Lambda(lambda x: K.sum(x, axis=2))(cnn3)
    print(cnn3.shape)
    
    hidden1=Dense(400,activation="relu")(cnn3)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    out = Dense(len(labels), activation='sigmoid')(hidden4)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy"]) 

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


(None, 30, 1)
(None, 30, 1)
(None, 10, 1)
(None, 10)
Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
permute_2 (Permute)          (None, 300, 300)          0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 300, 300, 1)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 296, 1, 1)         1501      
_________________________________________________________________
lambda_7 (Lambda)            (None, 296, 1)            0         
________________________________________________________

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 5 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_3cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/250
405/405 [==============================] - 55s 134ms/step - loss: 0.4371 - accuracy: 0.0197 - val_loss: 0.1157 - val_accuracy: 0.0135

Epoch 00001: val_loss improved from inf to 0.11573, saving model to best_3cnn_4fc.h5
Epoch 2/250
405/405 [==============================] - 54s 135ms/step - loss: 0.1123 - accuracy: 0.0292 - val_loss: 0.1120 - val_accuracy: 0.0148

Epoch 00002: val_loss improved from 0.11573 to 0.11202, saving model to best_3cnn_4fc.h5
Epoch 3/250
405/405 [==============================] - 55s 135ms/step - loss: 0.1109 - accuracy: 0.0331 - val_loss: 0.1110 - val_accuracy: 0.0169

Epoch 00003: val_loss improved from 0.11202 to 0.11097, saving model to best_3cnn_4fc.h5
Epoch 4/250
405/405 [==============================] - 54s 134ms/step - loss: 0.1087 - accuracy: 0.0370 - val_loss: 0.1102 - val_accuracy: 0.0564

Epoch 00004: val_loss improved from 0.11097 to 0.11019, saving model to best_3cnn_4fc.h5
Epoch 5/250
405/405 [==============================] - 54s 1

# ***If the model is generated before:***

In [ ]:
model = load_model('best_3cnn_4fc.h5')
# model.save('CNN_1_no_binary.h5')

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
print(pred[0])#example

[1.31797791e-03 4.16785479e-04 2.26811608e-06 2.61276960e-04
 2.29895115e-04 4.80353832e-04 1.68446586e-05 2.99449439e-05
 6.32633237e-06 4.35888767e-04 3.05622816e-04 3.70409467e-11
 1.19592573e-06 1.45673752e-04 4.82328032e-05 9.96172428e-04
 1.42338872e-03 1.70068473e-01 4.97731566e-03 2.03025341e-03
 7.98917413e-02 1.48856640e-03 2.05814838e-04 5.04848361e-03
 3.04391980e-03 1.04603171e-02 1.92552805e-04 2.09915638e-03
 1.17431438e-04 3.05207968e-02 1.97535753e-03 3.57646644e-02
 5.89382648e-03 2.54356861e-03 3.00536454e-02 3.91513109e-04
 1.20496195e-04 1.16161095e-06 2.33818591e-02 9.97573137e-04
 1.29052401e-02 4.69183922e-03 5.65588474e-04 1.29723549e-03
 2.70754099e-04 1.49995089e-04 6.51270151e-04 4.70280647e-04
 1.02825325e-04 9.30339098e-03 2.43358973e-05 5.35402796e-05
 2.23636925e-02 5.56443119e-05 8.09103251e-04 6.09904528e-04
 1.70326233e-03 6.58306479e-03 1.41492128e-01 8.78146291e-03
 9.11315978e-02 1.15367174e-02 8.13841820e-04 2.31206417e-04
 4.45336103e-03 2.469986

In [ ]:
y_pred=[]
measure = np.mean(pred[0]) + 1.15*np.sqrt(np.var(pred[0]))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
measure

0.05155983436852693

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.025278293135435993
              precision    recall  f1-score   support

           0       0.11      0.42      0.17       126
           1       0.12      0.59      0.20       156
           2       0.11      0.71      0.19       131
           3       0.00      0.00      0.00        12
           4       0.40      0.84      0.54        94
           5       0.00      0.00      0.00        11
           6       0.11      0.66      0.19       140
           7       0.23      0.33      0.27        36
           8       0.02      0.03      0.03        32
           9       0.06      0.24      0.10       105
          10       0.09      0.53      0.15       142
          11       0.43      0.91      0.59        67
          12       0.63      0.97      0.76       125
          13       1.00      0.06      0.11        17
          14       0.00      0.00      0.00        20
          15       0.09      0.58      0.16       146
          16       0.14      0.60      0.23       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.21335807050092764
              precision    recall  f1-score   support

           0       0.30      0.40      0.34       123
           1       0.16      0.39      0.23       132
           2       0.39      0.57      0.46       149
           3       0.30      0.32      0.31        19
           4       0.96      0.93      0.94        82
           5       0.00      0.00      0.00        11
           6       0.30      0.59      0.40       152
           7       0.54      0.15      0.23        48
           8       0.18      0.15      0.16        34
           9       0.17      0.15      0.16       123
          10       0.32      0.43      0.37       161
          11       0.85      0.97      0.91        72
          12       0.90      0.97      0.94       142
          13       0.43      0.20      0.27        15
          14       0.00      0.00      0.00        18
          15       0.21      0.47      0.29       148
          16       0.38      0.66      0.48       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
